In [ ]:
import sqlite3
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain_community.llms import Ollama
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import FakeEmbeddings 
from langchain.chains import RetrievalQA

# --- 1. CONFIGURACIÓN DE LA BASE DE DATOS Y RAG ---

DB_NAME = "inventario.db"

def init_db():
    """Crea la base de datos relacional y las tablas de productos/movimientos."""
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    
    # 1. Crear tabla de productos (Stock actual)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS productos (
            id_producto INTEGER PRIMARY KEY AUTOINCREMENT,
            nombre TEXT NOT NULL UNIQUE,
            stock_actual INTEGER NOT NULL DEFAULT 0,
            ubicacion TEXT
        )
    """)
    
    # 2. Crear tabla de movimientos (para auditoría)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS movimientos (
            id_movimiento INTEGER PRIMARY KEY AUTOINCREMENT,
            id_producto INTEGER,
            tipo_movimiento TEXT CHECK(tipo_movimiento IN ('ENTRADA', 'SALIDA')) NOT NULL,
            cantidad INTEGER NOT NULL,
            fecha_movimiento TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (id_producto) REFERENCES productos(id_producto)
        )
    """)

    # 3. Insertar datos iniciales
    productos_iniciales = [
        ('Tornillo M4 x 10mm', 150, 'A1-03'),
        ('Martillo Bicolor 500g', 25, 'B2-01'),
        ('Taladro Perforador XL', 10, 'C1-05')
    ]
    cursor.executemany("INSERT OR IGNORE INTO productos (nombre, stock_actual, ubicacion) VALUES (?, ?, ?)", productos_iniciales)
    conn.commit()
    conn.close()
    print(f"Base de datos relacional '{DB_NAME}' inicializada con éxito.")

# 1.2 Base de Conocimiento RAG (Simulación - Fines de demostración)
DOCUMENTOS_RAG = [
    "La política de devoluciones establece un plazo de 30 días con el embalaje original.",
    "El procedimiento para daños en el envío requiere llenar el Formulario 34B y notificar al proveedor en 48 horas.",
]
# Usa FakeEmbeddings y FAISS para simular la Base Vectorial
vectorstore = FAISS.from_texts(DOCUMENTOS_RAG, FakeEmbeddings(size=5))
retriever = vectorstore.as_retriever()

# --- 2. HERRAMIENTAS (Function Calling) ---

@tool
def consultar_stock(nombre_producto: str) -> str:
    """Usa esta herramienta para obtener la cantidad de stock actual de un producto por su nombre."""
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute("SELECT stock_actual, ubicacion FROM productos WHERE nombre = ?", (nombre_producto,))
    resultado = cursor.fetchone()
    conn.close()

    if resultado:
        stock, ubicacion = resultado
        return f"El stock actual de '{nombre_producto}' es {stock}. Se encuentra en la ubicación: {ubicacion}."
    return f"Error: Producto '{nombre_producto}' no encontrado en el inventario."

@tool
def actualizar_stock(nombre_producto: str, delta_cantidad: int) -> str:
    """
    Usa esta herramienta para AÑADIR o RESTAR stock de un producto.
    Si recibes unidades, 'delta_cantidad' es positivo. Si vendes/restas, es negativo.
    """
    if delta_cantidad == 0:
        return "Advertencia: La cantidad de actualización es cero. No se realizó ningún cambio."
        
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    
    # 1. Obtener ID del producto antes de la actualización (necesario para la tabla de movimientos)
    cursor.execute("SELECT id_producto FROM productos WHERE nombre = ?", (nombre_producto,))
    producto_id_result = cursor.fetchone()
    if not producto_id_result:
        conn.close()
        return f"Error: No se pudo actualizar. Producto '{nombre_producto}' no encontrado."
    
    producto_id = producto_id_result[0]

    # 2. Actualizar stock (Transacción principal)
    cursor.execute("UPDATE productos SET stock_actual = stock_actual + ? WHERE id_producto = ?", (delta_cantidad, producto_id))
    
    # 3. Registrar el movimiento (Auditoría)
    tipo = 'ENTRADA' if delta_cantidad > 0 else 'SALIDA'
    cursor.execute("INSERT INTO movimientos (id_producto, tipo_movimiento, cantidad) VALUES (?, ?, ?)", 
                   (producto_id, tipo, abs(delta_cantidad)))
    
    # 4. Obtener el nuevo stock para confirmar (NLG)
    cursor.execute("SELECT stock_actual FROM productos WHERE id_producto = ?", (producto_id,))
    nuevo_stock = cursor.fetchone()[0]
    conn.commit()
    conn.close()

    accion = "añadido" if delta_cantidad > 0 else "reducido"
    return f"Inventario actualizado: Se han {accion} {abs(delta_cantidad)} unidades de '{nombre_producto}'. El nuevo stock es {nuevo_stock}."

# 2.3 Herramienta RAG (Implementada como una herramienta del Agente)
# El LLM decidirá si usa esta función para responder preguntas de documentos.
@tool
def consulta_documentos(pregunta: str) -> str:
    """Usa esta herramienta para responder preguntas sobre políticas, manuales, o procedimientos que NO son datos de stock."""
    # En un sistema real, el LLM usaría un chain para generar la respuesta RAG.
    # Aquí simulamos la respuesta para no depender del LLM en el RAG chain
    if "política" in pregunta.lower() or "procedimiento" in pregunta.lower():
        # Usamos el retriever para buscar en los documentos simulados
        docs = retriever.get_relevant_documents(pregunta)
        contexto = "\n".join([doc.page_content for doc in docs])
        
        # En un entorno real, este contexto sería pasado a Llama para el NLG
        return f"CONTEXTO RECUPERADO:\n{contexto}"
    
    return "La información requerida no se encontró en la base de conocimientos documentales."


# --- 3. CONFIGURACIÓN DEL AGENTE (LLM/Ollama) ---

# 3.1 Integración del LLM real (Llama 3)
try:
    # Asegúrate de que Ollama esté corriendo en http://localhost:11434
    llm = Ollama(model="llama3", base_url="http://localhost:11434")
except Exception as e:
    print(f"Error al conectar con Ollama. ¿Está corriendo Llama 3? -> {e}")
    # Usar un LLM simulado si falla la conexión (para testing)
    from langchain_community.llms import FakeListLLM
    llm = FakeListLLM(responses=["Simulación: Llamada a herramienta exitosa"])


# 3.2 Definición del Prompt y Herramientas
SYSTEM_PROMPT = """
Eres un Agente de Gestión de Inventario inteligente y profesional.
Tu principal tarea es asistir al empresario.
1. Utiliza las herramientas (consultar_stock, actualizar_stock) para cualquier solicitud de inventario en tiempo real.
2. Utiliza la herramienta 'consulta_documentos' para cualquier pregunta sobre políticas, manuales o procedimientos.
3. Responde siempre de forma clara, profesional y amigable, basándote en la salida de las herramientas.
"""

tools = [consultar_stock, actualizar_stock, consulta_documentos]
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYSTEM_PROMPT),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# 3.3 Creación y Ejecución del Agente
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


# --- 4. PRUEBAS DE FUNCIONAMIENTO ---

if __name__ == "__main__":
    init_db()
    print("\n" + "="*50)
    print("🤖 CHATBOT DE INVENTARIO HÍBRIDO CON LLAMA 3 (OLLAMA)")
    print("="*50 + "\n")

    # ----------------------------------------------------
    # PRUEBA 1: Función Calling (Actualización con entrada)
    # ----------------------------------------------------
    print(">>> PRUEBA 1: Actualizar stock de Tornillo M4 (ENTRADA)")
    response_update = agent_executor.invoke({"input": "Acabamos de recibir 100 unidades más de Tornillo M4 x 10mm, añádelas."})
    print(f"\nRespuesta del Chatbot:\n{response_update['output']}\n")

    # ----------------------------------------------------
    # PRUEBA 2: Función Calling (Consulta)
    # ----------------------------------------------------
    print(">>> PRUEBA 2: Consulta de stock de Martillo Bicolor (CONSULTA)")
    response_query = agent_executor.invoke({"input": "¿Cuántos Martillos Bicolor 500g me quedan?"})
    print(f"\nRespuesta del Chatbot:\n{response_query['output']}\n")

    # ----------------------------------------------------
    # PRUEBA 3: RAG (Consulta de Conocimiento Estático)
    # ----------------------------------------------------
    print(">>> PRUEBA 3: Consulta sobre una política (RAG)")
    response_rag = agent_executor.invoke({"input": "¿Cuál es el procedimiento que debemos seguir al recibir un envío con daños?"})
    print(f"\nRespuesta del Chatbot:\n{response_rag['output']}\n")